### Bibliotecas

In [0]:
#Importar Bibliotecas
from pyspark.sql.functions import *
from pyspark.sql.types import *
import great_expectations as gx
import json

In [0]:
pip install great_expectations

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


### Camada Bronze: 
Extração dos dados brutos, remoção dos duplicados e conversão para um arquivo parquet

In [0]:
#caminho para os dados brutos
path_listing_raw = "/FileStore/tables/raw/listings_csv.gz"
path_reviews_raw = "/FileStore/tables/raw/reviews_csv.gz"
path_calendar_raw = "/FileStore/tables/raw/calendar_csv.gz"

#Extração dos dados brutos em um DataFrame
#Remoção de linhas duplicadas
df_listings = spark.read.option("multiline", "true").option("quote",'"').option("escape", '"').csv(path_listing_raw, header=True)
df_listings = df_listings.dropDuplicates()

df_reviews = spark.read.option("multiline", "true").option("quote",'"').option("escape", '"').csv(path_reviews_raw, header=True)
df_reviews = df_reviews.dropDuplicates()

df_calendar = spark.read.option("multiline", "true").option("quote",'"').option("escape", '"').csv(path_calendar_raw, header=True)
df_calendar = df_calendar.dropDuplicates()

#Carga dos dados para a primeira camada: Bronze
df_listings.write.parquet("/FileStore/tables/bronze/listings_bronze.parquet")
df_reviews.write.parquet("/FileStore/tables/bronze/reviews_bronze.parquet")
df_calendar.write.parquet("/FileStore/tables/bronze/calendar_bronze.parquet")

### Camada Silver: 
Listings

In [0]:
#Extração dos dados listings da camada bronze
df_listings = spark.read.parquet("/FileStore/tables/bronze/listings_bronze.parquet", header=True)

#Identificação das colunas da tabela listings

long_type = ["id", "scrape_id", "host_id"]
data = ["last_scraped","host_since", "calendar_last_scraped", "first_review", "last_review"]
inteiro = ["host_listings_count", "host_total_listings_count", "accommodates", "bathrooms","bedrooms", "beds", "minimum_nights", "maximum_nights","minimum_minimum_nights", "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights", "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", "availability_30", "availability_60", "availability_90", "availability_365", "number_of_reviews", "number_of_reviews_ltm", "number_of_reviews_l30d", "calculated_host_listings_count", "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms" ]
booleano = ["host_is_superhost", "host_has_profile_pic", "host_identity_verified", "has_availability", "instant_bookable" ]
lista = ["host_verifications", "amenities"]
decimal = ["host_response_rate", "host_acceptance_rate", "latitude", "longitude", "price", "reviews_per_month",  "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value" ]

#Tratamento na camada silver: Listings
#Tratamento da coluna price

df_listings = df_listings.withColumn("price", regexp_replace("price", "\\$", "")).withColumn("price", regexp_replace("price", ",", "")).withColumn("price", regexp_replace("price", " ", ""))

#Transformação para LongType os ids
for coluna in long_type:
    df_listings = df_listings.withColumn(coluna, df_listings[coluna].cast(LongType()))

#Transformação para data
for coluna in data:
    df_listings = df_listings.withColumn(coluna, to_date(df_listings[coluna], "yyyy-MM-dd"))

#Tranformação para inteiro
for coluna in inteiro:
    df_listings = df_listings.withColumn(coluna, df_listings[coluna].cast("integer"))

#Tranformação para float
for coluna in decimal:
    df_listings = df_listings.withColumn(coluna, df_listings[coluna].cast("float"))

#Transformação para Booleano
for coluna in booleano:
    df_listings = df_listings.withColumn(coluna, when(df_listings[coluna]== "t", True).when(df_listings[coluna]=="f", False).otherwise(None).cast("boolean"))

#Tranformação para lista
for coluna in lista:
    df_listings = df_listings.withColumn(coluna, split(df_listings[coluna], ","))

#Importar os dados tratados para um arquivo parquet
df_listings.write.parquet("/FileStore/tables/silver/listings_silver.parquet")

### Camada Silver:
Reviews

In [0]:
#Extração dos dados reviews da camada bronze
df_reviews = spark.read.parquet("/FileStore/tables/bronze/reviews_bronze.parquet")

#Transformação do formato da coluna "date" de string para date
df_reviews= df_reviews.withColumn("date", to_date("date", "yyyy-MM-dd"))
strings = ["listing_id", "id", "reviewer_id"]

#Tranformação das colunas de id em LongType
for coluna in strings:
    df_reviews = df_reviews.withColumn(coluna, df_reviews[coluna].cast(LongType()))


#Carga dos dados para um arquivo parquet da camada silver
df_reviews.write.parquet("/FileStore/tables/silver/reviews_silver.parquet")


### Camada Silver:
Calendar

In [0]:
#Extração dos dados calendar da camada bronze
df_calendar = spark.read.parquet("/FileStore/tables/bronze/calendar_bronze.parquet")

#Transformação do formato da coluna "date" de string para date
df_calendar= df_calendar.withColumn("date", to_date("date", "yyyy-MM-dd"))

#Tranformação da coluna de id em LongType
df_calendar = df_calendar.withColumn("listing_id", df_calendar["listing_id"].cast(LongType()))

#Tranformação da coluna price
df_calendar = df_calendar.withColumn("price", regexp_replace("price", "\\$", "")).withColumn("price", regexp_replace("price", ",", "")).withColumn("price", regexp_replace("price", " ", "").cast("float"))
#Transformação das colunas maximum e minimum nights de string para integer
df_calendar = df_calendar.withColumn("minimum_nights", df_calendar["minimum_nights"].cast("integer"))
df_calendar = df_calendar.withColumn("maximum_nights", df_calendar["maximum_nights"].cast("integer"))

#Tranformação da coluna "available" de string para booleano
df_calendar = df_calendar.withColumn("available", when(df_calendar["available"]== "t", True).when(df_calendar["available"]=="f", False).cast("boolean"))

#Carga dos dados para um arquivo parquet na camada silver
df_calendar.write.parquet("/FileStore/tables/silver/calendar_silver.parquet")

### Great Expectations:
Listings

In [0]:
#Obtenção do contexto Great Expectations
context = gx.get_context()

#Adição de fonte de dados para o contexto
data_source = context.data_sources.add_spark("teste_spark")

#Adição de ativo de dados 
data_asset = data_source.add_dataframe_asset(name="spark dataframe asset")

#Definição de batch
batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")

#Obtenção do batch para o dataframe listings
batch = batch_definition.get_batch(batch_parameters={"dataframe": df_listings})

#Criação e adição de Expectativa suite 
suite = gx.ExpectationSuite(name="suite_teste")
suite = context.suites.add(suite)

#Great Expectations
#Para cada coluna nos diferentes tipos, adiciona expectativas de tipos de dados na suite

for coluna in long_type:
  expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="LongType")
  suite.add_expectation(expectation)
for coluna in data:
  expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="DateType")
  suite.add_expectation(expectation)

for coluna in decimal:
  expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="FloatType")
  suite.add_expectation(expectation)
for coluna in inteiro:
  expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="IntegerType")
  suite.add_expectation(expectation)
for coluna in booleano:
  expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="BooleanType")
  suite.add_expectation(expectation)

#Verifica as expectativas para a coluna Price que deve ter como valor minimo 0 
expectation = gx.expectations.ExpectColumnValuesToBeBetween(column="price", min_value=0, strict_min=True)
suite.add_expectation(expectation)

#Cria uma definição de validação no Great Expectations, vinculando o conjunto de dados e a Expectation Suite
validation_definition = gx.ValidationDefinition(data=batch_definition, suite=suite, name="validacao de dados listings")

#Executa a validação usando a definição criada, passando o DataFrame 'df_listings' como parâmetro para os dados a serem validados
validation_definition.run(batch_parameters={"dataframe":df_listings})


Calculating Metrics:   0%|          | 0/14 [00:00<?, ?it/s]

{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "batch_id": "teste_spark-spark dataframe asset",
          "column": "id",
          "type_": "LongType"
        },
        "meta": {},
        "id": "ebb279e3-a37e-42a8-a751-f4982053f86b"
      },
      "result": {
        "observed_value": "LongType"
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "batch_id": "teste_spark-spark dataframe asset",
          "column": "scrape_id",
          "type_": "LongType"
        },
        "meta": {},
        "id": "b4e60e6d-bbab-4c74-9ddb-f879a933137c"
      },
      "result": {
        "observed_value": "

### Great Expectations:
Reviews

In [0]:
#Obtenção do contexto Great Expectations
context = gx.get_context()

#Adição de fonte de dados para o contexto
data_source = context.data_sources.add_spark("teste_spark_reviews")

#Adição de ativo de dados 
data_asset = data_source.add_dataframe_asset(name="spark dataframe asset")

#Definição de batch
batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")

#Obtenção do batch para o dataframe reviews
batch = batch_definition.get_batch(batch_parameters={"dataframe": df_reviews})

#Criação e adição de Expectativa suite 
suite = gx.ExpectationSuite(name="suite_teste_reviews")
suite = context.suites.add(suite)

#Great Expectations
#Para cada coluna no tipo LongType, adiciona expectativas de tipos de dados na suite
for coluna in strings:
  expectation = gx.expectations.ExpectColumnValuesToBeOfType(column=coluna, type_="LongType")
  suite.add_expectation(expectation)

#Cria uma definição de validação no Great Expectations, vinculando o conjunto de dados e a Expectation Suite
validation_definition = gx.ValidationDefinition(data=batch_definition, suite=suite, name="validacao de dados reviews")

#Executa a validação usando a definição criada, passando o DataFrame 'df_reviews' como parâmetro para os dados a serem validados
validation_definition.run(batch_parameters={"dataframe":df_reviews})


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "batch_id": "teste_spark_reviews-spark dataframe asset",
          "column": "listing_id",
          "type_": "LongType"
        },
        "meta": {},
        "id": "90b87a47-b0ed-498a-bcc8-75e8c798521a"
      },
      "result": {
        "observed_value": "LongType"
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "batch_id": "teste_spark_reviews-spark dataframe asset",
          "column": "id",
          "type_": "LongType"
        },
        "meta": {},
        "id": "79d097dc-8c6d-4f7d-9532-ae8c405e5dee"
      },
      "result": {
        "ob

### Great Expectations:
Calendar

In [0]:
#Obtenção do contexto Great Expectations
context = gx.get_context()

#Adição de fonte de dados para o contexto
data_source = context.data_sources.add_spark("teste_spark_calendar")

#Adição de ativo de dados 
data_asset = data_source.add_dataframe_asset(name="spark dataframe asset")

#Definição de batch
batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")

#Obtenção do batch para o dataframe Calendar
batch = batch_definition.get_batch(batch_parameters={"dataframe": df_calendar})

#Criação e adição de Expectativa suite 
suite = gx.ExpectationSuite(name="suite_teste_calendar")
suite = context.suites.add(suite)

#Great Expectations
#Para cada coluna nos diferentes tipos, adiciona expectativas de tipos de dados na suite
expectation = gx.expectations.ExpectColumnValuesToBeOfType(column="date", type_="DateType")
suite.add_expectation(expectation)

expectation = gx.expectations.ExpectColumnValuesToBeOfType(column="listing_id", type_="LongType")
suite.add_expectation(expectation)

expectation = gx.expectations.ExpectColumnValuesToBeOfType(column="minimum_nights", type_="IntegerType")
suite.add_expectation(expectation)

expectation = gx.expectations.ExpectColumnValuesToBeOfType(column="maximum_nights", type_="IntegerType")
suite.add_expectation(expectation)

expectation = gx.expectations.ExpectColumnValuesToBeOfType(column="available", type_="BooleanType")
suite.add_expectation(expectation)

#Verifica as expectativas para a colunas que devem ter como valor minimo 0 
expectation = gx.expectations.ExpectColumnValuesToBeBetween(column="minimum_nights", min_value=0, strict_min=True)
suite.add_expectation(expectation)

#Cria uma definição de validação no Great Expectations, vinculando o conjunto de dados e a Expectation Suite
expectation = gx.expectations.ExpectColumnValuesToBeBetween(column="maximum_nights", min_value=0, strict_min=True)
suite.add_expectation(expectation)

#Executa a validação usando a definição criada, passando o DataFrame 'df_reviews' como parâmetro para os dados a serem validados
validation_definition = gx.ValidationDefinition(data=batch_definition, suite=suite, name="validacao de dados Calendar")
validation_definition.run(batch_parameters={"dataframe":df_calendar})

Calculating Metrics:   0%|          | 0/23 [00:00<?, ?it/s]

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "batch_id": "teste_spark_calendar-spark dataframe asset",
          "column": "date",
          "type_": "DateType"
        },
        "meta": {},
        "id": "73832c9f-ae1d-4f7a-8a80-eeb1959fec0b"
      },
      "result": {
        "observed_value": "DateType"
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "batch_id": "teste_spark_calendar-spark dataframe asset",
          "column": "listing_id",
          "type_": "LongType"
        },
        "meta": {},
        "id": "2317658f-853d-4856-a239-8047c09d8707"
      },
      "result": {
       

### Camada Gold

In [0]:
#Extração dos dados da camda silver pós tratamento
df_listings = spark.read.parquet("/FileStore/tables/silver/listings_silver.parquet")
df_reviews = spark.read.parquet("/FileStore/tables/silver/reviews_silver.parquet")
df_calendar = spark.read.parquet("/FileStore/tables/silver/calendar_silver.parquet")

#Salva os dados extraidos da camada silver em tabela do databricks para posterior consulta em SQL
df_listings.write.mode("overwrite").saveAsTable("listings")
df_reviews.write.mode("overwrite").saveAsTable("reviews")
df_calendar.write.mode("overwrite").saveAsTable("calendar")

### Análise de Preço por Localização

In [0]:
#Salvamento da consulta sql em um dataframe
df_localizacao = spark.sql("SELECT ROUND(avg(price),2) AS average_price, neighbourhood AS city, neighbourhood_cleansed AS neighbourhood, ROUND(avg(review_scores_location),2) AS location_rating, ROUND(avg( review_scores_rating),2) AS review_rating from listings GROUP BY neighbourhood, neighbourhood_cleansed ORDER BY city DESC")

#Salvar o dataframe como uma tabela do databricks
df_localizacao.write.mode("overwrite").saveAsTable("preco_localizacao")

### Análise de Preço por tipo de propriedade

In [0]:
#Salvamento da consulta sql em um dataframe
df_propriedade = spark.sql("SELECT ROUND(avg(price),2) AS average_price, property_type, room_type, accommodates,  bathrooms, bedrooms, beds, ROUND(avg( review_scores_rating),2) AS review_rating from listings GROUP BY property_type, room_type,accommodates, bathrooms, bedrooms, beds ORDER BY room_type") 

#Salvar o dataframe como uma tabela do databricks
df_propriedade.write.mode("overwrite").saveAsTable("preco_tipo_propriedade")

### Análise de Host

In [0]:
#Salvamento da consulta sql em um dataframe
df_host = spark.sql("SELECT host_id, host_name, host_since, host_location, host_neighbourhood, avg(host_response_time) AS response_time, avg(host_response_rate) AS response_rate, avg(host_acceptance_rate) AS acceptance_rate, host_is_superhost, host_identity_verified FROM listings GROUP BY host_id, host_name, host_since, host_location, host_neighbourhood, host_is_superhost, host_identity_verified ORDER BY  host_id")

#Salvar o dataframe como uma tabela do databricks
df_host.write.mode("overwrite").saveAsTable("analise_host")

### Disponibilidade e Calendário

In [0]:
#Salvamento da consulta sql em um dataframe
df_disponibilidade = spark.sql("SELECT l.id ,c.date, l.availability_30, l.availability_60, l.availability_90, l.availability_365, c.minimum_nights AS minimun_nigths, c.maximum_nights AS maximum_nights FROM listings l JOIN calendar c on l.id = c.listing_id ORDER BY id")

#Salvar o dataframe como uma tabela do databricks
df_disponibilidade.write.mode("overwrite").saveAsTable("disponibilidade")